<a href="https://colab.research.google.com/github/ExtractTable/ExtractTable-py/blob/master/example-code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installation

In [ ]:
!pip install -U ExtractTable

# 2. Import and check version

In [ ]:
from ExtractTable import ExtractTable

In [ ]:
print(ExtractTable.VERSION)

# 3. Create Session & Validate API Key


## 3.1 **Create Session** with your API Key

In [ ]:
api_key = YOUR_APIKEY_HERE

In [ ]:
et_sess = ExtractTable(api_key)

In [ ]:
# et_sess.__dict__

## 3.2 **Validate** the Key and check the plan usage

In [ ]:
usage = et_sess.check_usage()

*If there is no error encountered in the above cell, it means we have a valid API key. Now, lets get started by checking the usage and trigger the file for processing*

In [ ]:
# et_sess.server_response

In [ ]:
print(usage)

## 3.3 Check Usage Details

**credits**: Total number credits attached to the API Key

**queued** : Number of triggered jobs that are still processing in the queue

**used**   : Number of credits already used 

# 4. Trigger the extraction process

> Note: We will use the session, `et_sess`, created earlier in step 3.1, to save the session data and retrieve when needed

## 4.1 Accepted Input Types

**Allowed input formats** are:
- Image
  - JPG/JPEG
  - PNG
- PDF
  - Text PDF
  - Scan PDF
  - Image PDF


**Input Location Options**
- Location can be a file from the local drive
- Accessible remote URL - *the file object will be locally downloaded and deleted once sent to the process*

In [ ]:
# image_location = "local_image_path_OR_remote_image_url_with_tables"
# image_location = r'samples/BlurryImage.jpg'
image_location = "https://raw.githubusercontent.com/ExtractTable/ExtractTable-py/master/samples/QualityImage.jpg"

## 4.2 Process an <ins>IMAGE</ins> Input


In [ ]:
table_data = et_sess.process_file(filepath=image_location, output_format="df")

In [ ]:
table_data

## 4.3 Process a <ins>PDF</ins> Input

In [ ]:
# pdf_location = "local_image_path_OR_remote_image_url_with_tables"
# pdf_location = r'samples/BlurryImage.jpg'
pdf_location = "https://raw.githubusercontent.com/ExtractTable/ExtractTable-py/master/samples/QualityImage.jpg"

In [ ]:
table_data = et_sess.process_file(filepath=Location_of_PDF_with_Tables, pages="all", output_format="df")

Below are the sample values ```pages``` accepts **string** type



| pages  	| Explanation                                     	|
|----------	|-------------------------------------------------	|
| "1"      	| [Default] considers only 1st page of the PDF    	|
| "1,3,5"  	| considers only 1st, 3rd and 5th page of the PDF 	|
| "1, 3-5" 	| considers 1st, 3rd, 4th and 5th page of the PDF 	|
| "all"    	| considers complete PDF                          	|

## 4.4 Table Output options

> By default, the `process_file()` returns **only** the table data. Output depends on the `output_format` , explained below

Explore the available options with `ExtractTable._OUTPUT_FORMATS`

| output_format 	| Explanation                                	|
|---------------	|--------------------------------------------	|
| "df"          	| [Default] Array of Pandas dataframes        	|
| "dataframe"   	| same as "df"; Array of Pandas dataframes    	|
| "json"        	| JSON data with index orientation           	|
| "dict"        	| Similar to JSON data but python dictionary 	|
| "csv"         	| Array of locally saved CSV file locations   	|
| "xlsx"         	| To save multiple tables as sheets into a single excel	|
| "excel"         | same as "xlsx"; output is an array of excel location	|


Default output is an array of pandas dataframes, with which you can change to any other format like excel, html etc. Follow https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html


## 4.5 Explore session objects

> **Explore** all objects of the latest file processing with `et_sess.__dict__.
keys()` - Depends on the plan type of your API Key

In [ ]:
et_sess.__dict__.keys()

Based on the API Key PLAN type, the et_sess contains below objects

| Object          	| Explanation                                                                            	|
|-----------------	|----------------------------------------------------------------------------------------	|
| api_key         	| Your API Key                                                                           	|
| _session        	| Session data of the **latest** performed request/action                                          	|
| input_filename  	| Name of the processed input file |
| ServerResponse  	| Complete ServerResponse, along with response code and headers                          	|
| server_response 	| complete server response content; equivalent to `ServerResponse.json()`                	|
| JobStatus       	| Job Status of the triggered process                                                    	|
| Pages           	| Number of pages in the input; also number of credits consumed on the triggered process 	|
| Tables          	| Tabular Data in JSON format with index orientation; ordered table wise                 	|
| Lines           	| Text Data in JSON format, ordered page wise                                            	|

## 4.6 Save Table & Text to LOCAL

```python
et_sess.save_output(output_folder, output_format="csv")
```
`output_format` param is relavant only for the table data, with options "csv" or "xlsx"


> Note: As the `et_sess` contains the latest action performed, make sure this call is right after the `process_file()`

# 5. Explore the Output

## 5.1 Output Structure

> **Understand the output**: The response of a triggered job is a JSON object in the below format. 

Note that the response depends on the plan type of the API Key.


```javascript
{
    "JobStatus": <string>,                              # Status of the triggered Process  @ JOB-LEVEL
    "Pages": <integer>,                                 # Number of pages processed in this request @ PAGE-LEVEL
    "Tables": [<list of key-value objects of table>     # List of all tables found @ TABLE-LEVEL
        {
            "Page": <integer>,                              ## Page number in which this table is found
            "CharacterConfidence": <float>,                 ## Accuracy of Characters recognized from the input-page
            "LayoutConfidence": <float>,                    ## Accuracy of table layout's design decision
            "TableJson": <dict>,                            ## Table Cell Text in key-value format with index orientation - {row#: {col#: <str>}}
            "TableCoordinates": <dict>,                     ## Top-left & Bottom-right Cell Coordinates - {row#: {col#: <list(x1,y1,x2,y2)>}}
            "TableConfidence": <dict>                       ## Cell level accuracy of detected characters - {row#: {col#: <float>}}
        },
    {...}                                               ## ... more "Tables" objects
    ],
    "Lines": [<list of key-value objects>               # Pagewise Line details @ PAGE-LEVEL
        {
            "Page": <integer>,                          # Page number in which the lines are found
            "CharacterConfidence": <float>,             # Average Accuracy of all Characters recognized from the input-page
            "LinesArray": [
                <list of key-value objects of line>     # Ordered list of lines in this page @ LINE-LEVEL
                {
                    "Line": <str>,                          ## Detected text of the complete line
                    "WordsArray": [
                        <list of key-value objects>         ## Word level datails in this line @ WORD-LEVEL
                        {
                            "Conf": <float>,                    ### Accuracy of recognized characters of the word
                            "Word": <str>,                      ### Detected text of the word
                            "Loc": [x1, y1, x2, y2]             ### Top-left & Bottom-right coordinates, w.r.t the input-page width-height dimensions
                        },
                    {...}                                   ### More "WordsArray" objects
                    ]
                },
            {...}                                       ## More "LinesArray" objects
            ]
        },
    {...}                                               # More Pagewise "Lines" details
    ]
}
```

## 5.2 Output Details

Output objects are based on the API Key Plan type. Available plan types are 

**Purchased Plans**
*   "LITE"   - **only table data** in the output
*   "FULL"   - **table and text data** in the output
*   "EXTRA"  - **table, text data along with cell & word coordintates and character detection accuracy**

**Promotional Plans**: Any plan other than Purchased plans are promotional
*   "free_trial", "camelotpro" - these are promotional API Keys, gives only table data equivalent to "LITE" plan type


<br>
Output objects detail below



| Key Name 	| Parent 	| Type 	| Description 	| Availability 	|
|-	|-	|-	|-	|-	|
| JobStatus 	| Job 	| String 	| Status of the triggered process 	| ALL Plans 	|
| Pages 	| Job 	| Integer 	| Number of pages processed in the request 	| ALL Plans 	|
| Tables 	| Job 	| Array 	| List of all tables found 	| ALL Plans 	|
| Tables[0].Page 	| Table 	| Integer 	| Page number in which the table is found 	| ALL Plans 	|
| Tables[0].CharacterConfidence 	| Table 	| Decimal 	| Accuracy of Characters recognized from the image 	| ALL Plans 	|
| Tables[0].LayoutConfidence 	| Table 	| Decimal 	| Accuracy of table layout's design decision 	| ALL Plans 	|
| Tables[0].TableJson 	| Table 	| Json/dict 	| Table Cell Text in key-value format with index orientation - {row#: {col#: }} 	| ALL Plans 	|
| Tables[0].TableCoordinates 	| Table 	| Json/dict 	| Top-left & Bottom-right Cell Coordinates - {row#: {col#: }} 	| EXTRA Plan 	|
| Tables[0].TableConfidence 	| Table 	| Json/dict 	| Cell level accuracy of detected characters - {row#: {col#: }} 	| EXTRA Plan 	|
| Lines 	| Job 	| Array 	| List of page-wise lines text 	| FULL, EXTRA	|
| Lines[0].Page 	| Page 	| Integer 	| Page number in which the lines are found 	| Full Plan 	|
| Lines[0].CharacterConfidence 	| Page 	| Decimal 	| Average Accuracy of all Characters recognized from the input-page 	| Full Plan 	|
| Lines[0].LineArray 	| Page 	| Array 	| Ordered list of lines of the page 	|  	|
| Lines[0].LineArray[0].Line 	| Line 	| String 	| Detected text of the complete line 	| Full Plan 	|
| Lines[0].LineArray[0].WordsArray 	| Line 	| Array 	| Word level datails in this line 	| EXTRA Plan 	|
| Lines[0].LineArray[0].WordsArray[0].Conf 	| Word 	| Decimal 	| Accuracy of recognized characters of the word 	| EXTRA Plan 	|
| Lines[0].LineArray[0].WordsArray[0].Word 	| Word 	| String 	| Detected text of the word 	| EXTRA Plan 	|
| Lines[0].LineArray[0].WordsArray[0].Loc 	| Word 	| Array 	| Top-left & Bottom-right coordinates, w.r.t the input-page width-height dimensions 	| EXTRA Plan 	|

# 6. Make Corrections

> **Objective**: To ease corrections on the most common issues with the `MakeCorrections` module.

**Details:** The service relies on OCR (Optical Character Recognition) for character detection and deep learning models to detect tabular structures on the input. There may be a chance for merged rows or columns or incorrect type detections on low-quality inputs with a complex table layout or tightly packed columns. With those in mind, we want to offer the built-in service at the client-side to give control and ease in making corrections on the output. 


The module, `MakeCorrections`, currently supports below functionalities

| Functionality        	| Explanation                                    	|
|----------------------	|------------------------------------------------	|
| Split Merged Rows    	| Works well on cell values with no spaces       	|
| Split Merged Columns 	| Works well on cell values with no spaces       	|
| Fix Decimal Format   	| To fix thousand and decimal separators         	|
| Fix Date Format      	| To handle and modify incorrect date separators 	|
| Fix Characters       	| To fix incorrect character detections          	|


In [ ]:
# First things first lets import the module and prepare for corrections

from ExtractTable.common import MakeCorrections

corrections = MakeCorrections(et_resp=et_sess.server_response)

## 6.1 Split Merged Rows
        """
        To split the merged rows into possible multiple rows
        :return: reformatted list of dataframes
        """


In [ ]:
corrected_table_dataframes = corrections.split_merged_rows()

## 6.2 Split Merged Columns

        """
        To split the merged columns into possible multiple columns
        :param columns_idx: user preferred columns indices.
                Default loops through all columns to find numeric or decimal columns
        :param force_split: To force split through the columns
        :return: reformatted list of dataframes
        """


In [ ]:
corrected_table_dataframes = corrections.split_merged_columns()

## 6.3 Fix Decimal Format


        """
        To fix decimal and thousands separator values. Often commas as detected as period
        :param columns_idx: user preferred columns indices.
                Default loops through all columns to find numeric or decimal columns
        :param decimal_separator: preferred decimal separator
        :param thousands_separator: preferred thousands separator
        :param decimal_position: preferred decimal position
        :return: corrected list of dataframes
        """


In [ ]:
corrected_table_dataframes = corrections.fix_decimal_format(decimal_separator=".", thousands_separator=",")

## 6.4 Fix Date Format

        """
        To fix date formats of the column
        Eg: 12|1212020 as 12/12/2020
        :param columns_idx: user preferred columns indices.
                Default loops through all columns to find Date Columns
        :param delimiter: "/" or "-" whatelse you prefer
        :return: correted list of dataframes
        """


In [ ]:
corrected_table_dataframes = corrections.fix_date_format(delimiter="/")

## 6.5 Fix Characters

        """
        To replace incorrect character detections
        Eg: $123,45.0I as $123,45.01
        :param columns_idx: user preferred columns indices.
                Default loops through all columns to find Date Columns
        :param replace_ref: the replacement dictionary for reference
                Eg: {"I": "1"}
        :return: correted list of dataframes
        """


In [ ]:
corrected_table_dataframes = corrections.fix_characters(self, columns_idx=[0, 1], replace_ref={"I": "1", "S": "$"})

# 7. Helpful Code Snippets

Extra code snippets that are useful to perform some actions on the output. Based on the frequently asked questions.

## 7.1 Get text data

In [ ]:
# If your API Key supports "Lines"

all_page_lines = []
for each_page in et_sess.Lines:
  for each_line in each_page['LinesArray']:
    all_page_lines.append(each_line['Line'])
  
print("\n".join(all_page_lines))

## 7.2 All tables output to a single excel

In [ ]:
table_data = et_sess.process_file(filepath=Location_of_PDF_with_Tables, output_format="df", pages="all")
 
import pandas as pd
accumulate_all_dfs = pd.DataFrame()

for each_df in table_data:
    accumulate_all_dfs = accumulate_all_dfs.append(each_df, ignore_index=True)
    # print(each_df.shape, accumulate_all_dfs.shape)

print("Shape of all tables accumulated together is", accumulate_all_dfs.shape)


output_excel_location = <location_for_the_excel_output.xlsx>
# Save the accumulated output to a single excel file
accumulate_all_dfs.to_excel(output_excel_location, index=False, header=False)

# 8. View transactions

To view all the transactions that were triggered in the last 24 hours

In [ ]:
et_sess.view_transactions()

Return a list of transactions, with each record having below properties

|Property | Description |
|---|---|
|JobStatus | Status of the job|
|Pages | number of pages of the input; can also be considered as number of credits consumed|
|createdon | timestamp when the request was processed|
|requested_filename | Filename received in the request|
|txn_id | Unique identifier of the transaction, also referred as JobId to retrieve output via `get_result(JobId)`|

# 9. Support & Contact

Please do not hesitate to approach our developer team at pydevs@extracttable.com for any assitance needed or to report a bug